In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

# import matplotlib.pyplot as plt
import numpy as np

# from scipy import stats
from tqdm import tqdm

# from matplotlib.animation import FuncAnimation
from deslib.des.meta_des import METADES
from sklearn.model_selection import train_test_split
from skorch import NeuralNetClassifier

In [4]:
# print the version of DESLIB
import deslib

print(deslib.__version__)

0.3.5


In [5]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformation and load the MNIST dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

# Load MNIST dataset
mnist_train = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
mnist_test = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)  # 10 output classes for MNIST digits

    def forward(self, x):
        if x.dim() > 2:
            x = x.view(-1, 28 * 28)  # Flatten the image
        x = x.float()  # Convert to float
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [7]:
# Prepare the data loaders
def prepare_loaders(batch_size=64):
    # Get indices of training samples based on the label
    train_indices_01 = [
        i for i, label in enumerate(mnist_train.targets) if label == 0 or label == 1
    ]
    train_indices_23 = [
        i for i, label in enumerate(mnist_train.targets) if label == 2 or label == 3
    ]
    train_indices_45 = [
        i for i, label in enumerate(mnist_train.targets) if label == 4 or label == 5
    ]
    train_indices_67 = [
        i for i, label in enumerate(mnist_train.targets) if label == 6 or label == 7
    ]
    train_indices_89 = [
        i for i, label in enumerate(mnist_train.targets) if label == 8 or label == 9
    ]

    # Create Subsets
    subset_train_01 = Subset(mnist_train, train_indices_01)
    subset_train_23 = Subset(mnist_train, train_indices_23)
    subset_train_45 = Subset(mnist_train, train_indices_45)
    subset_train_67 = Subset(mnist_train, train_indices_67)
    subset_train_89 = Subset(mnist_train, train_indices_89)

    # Create DataLoader for all subsets
    loader_train_01 = DataLoader(subset_train_01, batch_size=64, shuffle=True)
    loader_train_23 = DataLoader(subset_train_23, batch_size=64, shuffle=True)
    loader_train_45 = DataLoader(subset_train_45, batch_size=64, shuffle=True)
    loader_train_67 = DataLoader(subset_train_67, batch_size=64, shuffle=True)
    loader_train_89 = DataLoader(subset_train_89, batch_size=64, shuffle=True)

    return (
        loader_train_01,
        loader_train_23,
        loader_train_45,
        loader_train_67,
        loader_train_89,
    )


def prepare_des_loaders(batch_size=64):
    train_indices, val_indices = train_test_split(
        list(range(len(mnist_train))), test_size=0.2, random_state=42
    )

    train_indices_01 = [
        i
        for i in train_indices
        if mnist_train.targets[i] == 0 or mnist_train.targets[i] == 1
    ]
    train_indices_23 = [
        i
        for i in train_indices
        if mnist_train.targets[i] == 2 or mnist_train.targets[i] == 3
    ]
    train_indices_45 = [
        i
        for i in train_indices
        if mnist_train.targets[i] == 4 or mnist_train.targets[i] == 5
    ]
    train_indices_67 = [
        i
        for i in train_indices
        if mnist_train.targets[i] == 6 or mnist_train.targets[i] == 7
    ]
    train_indices_89 = [
        i
        for i in train_indices
        if mnist_train.targets[i] == 8 or mnist_train.targets[i] == 9
    ]


    subset_train_01 = Subset(mnist_train, train_indices_01)
    subset_train_23 = Subset(mnist_train, train_indices_23)
    subset_train_45 = Subset(mnist_train, train_indices_45)
    subset_train_67 = Subset(mnist_train, train_indices_67)
    subset_train_89 = Subset(mnist_train, train_indices_89)

    subset_val = Subset(mnist_train, val_indices)

    loader_train_01 = DataLoader(subset_train_01, batch_size=64, shuffle=True)
    loader_train_23 = DataLoader(subset_train_23, batch_size=64, shuffle=True)
    loader_train_45 = DataLoader(subset_train_45, batch_size=64, shuffle=True)
    loader_train_67 = DataLoader(subset_train_67, batch_size=64, shuffle=True)
    loader_train_89 = DataLoader(subset_train_89, batch_size=64, shuffle=True)

    loader_val = DataLoader(subset_val, batch_size=64, shuffle=True)

    return (
        loader_train_01,
        loader_train_23,
        loader_train_45,
        loader_train_67,
        loader_train_89,
        loader_val,
    )

In [8]:
loader_01, loader_23, loader_45, loader_67, loader_89 = prepare_loaders()

In [9]:
(
    loader_train_01,
    loader_train_23,
    loader_train_45,
    loader_train_67,
    loader_train_89,
    loader_val,
) = prepare_des_loaders()

In [10]:
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=True)

In [11]:
def train_single_model(model, loaders, optimizer, criterion, epochs=5):
    model.train()
    for loader in tqdm(loaders):
        for epoch in range(epochs):
            for batch_idx, (data, target) in enumerate(loader):
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()


def train_skorch_model(model, loaders):
    for loader in loaders:
        dataset = loader.dataset.dataset
        indices = loader.dataset.indices

        # Ensure X is a float
        X = dataset.data[indices].float()

        # Ensure y is a long integer
        y = dataset.targets[indices].long()

        # partial_fit

        model.partial_fit(X, y)


def test_model(model, loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return correct / len(loader.dataset)


def train_des_ensemble(models, loader):
    original_dataset = loader.dataset.dataset
    indices = loader.dataset.indices

    # Reshape X to be 2-dimensional and convert to float
    X = original_dataset.data[indices].float()
    X = X.view(X.size(0), -1)  # Flatten the images into vectors

    # Ensure y is a long integer
    y = original_dataset.targets[indices].long()

    # Initialize and fit the METADES ensemble
    des = METADES(models)
    des.fit(X, y.long())  # y is also converted to long if necessary

    return des

In [12]:
# Define the model, optimizer and loss function
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model
train_single_model(
    model, [loader_01, loader_23, loader_45, loader_67, loader_89], optimizer, criterion
)

100%|██████████| 5/5 [00:14<00:00,  2.96s/it]


In [13]:
# test the model on the test set
test_model(model, test_loader)

0.1918

In [14]:
# define 5 classifiers for the DES ensemble
model_1 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)
model_2 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)

model_3 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)
model_4 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)
model_5 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)

model_6 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)

model_7 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)

model_8 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)

model_9 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.001,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)

model_10 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=0.01,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)

model_11 = NeuralNetClassifier(
    Net,
    max_epochs=5,
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    lr=1,
    device=device,
    iterator_train__shuffle=False,
    batch_size=64,
    train_split=None,
)

models = [
    model_1,
    model_2,
    model_3,
    model_4,
    model_5,
    model_6,
    model_7,
    model_8,
    model_9,
    model_10,
    model_11,
]

# train the models
for model in models:
    train_skorch_model(
        model,
        [
            loader_train_01,
            loader_train_23,
            loader_train_45,
            loader_train_67,
            loader_train_89,
        ],
    )

  epoch    train_loss     dur
-------  ------------  ------
      1        0.1052  0.1535
      2        0.0255  0.1467
      3        0.0097  0.1786
      4        0.0009  0.1575
      5        0.0000  0.2042
      6        1.0355  0.1531
      7        0.0560  0.1499
      8        0.0322  0.1495
      9        0.0243  0.1431
     10        0.0266  0.1498
     11        0.5614  0.1393
     12        0.0180  0.1309
     13        0.0100  0.1366
     14        0.0049  0.1370
     15        0.0015  0.1329
     16        0.4538  0.1447
     17        0.0039  0.1497
     18        0.0022  0.1434
     19        0.0009  0.1435
     20        0.0004  0.1419
     21        3.8988  0.1373
     22        1.5740  0.1366
     23        1.3408  0.1375
     24        1.1938  0.1358
     25        1.0934  0.1346
  epoch    train_loss     dur
-------  ------------  ------
      1        0.1625  0.1513
      2        0.0206  0.1502
      3        0.0166  0.1496
      4        0.0013  0.1500
      5   

In [15]:
# predict on the test set with model_1
y_pred = model_1.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of model_1: ", accuracy)

y_pred = model_2.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of model_2: ", accuracy)

y_pred = model_3.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of model_3: ", accuracy)

y_pred = model_4.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of model_4: ", accuracy)

y_pred = model_5.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of model_5: ", accuracy)

y_pred = model_6.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of model_6: ", accuracy)

y_pred = model_7.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of model_7: ", accuracy)

y_pred = model_8.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of model_8: ", accuracy)

y_pred = model_9.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)

print("Accuracy of model_9: ", accuracy)

y_pred = model_10.predict(test_loader.dataset.data.float())

accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)

print("Accuracy of model_10: ", accuracy)

Accuracy of model_1:  0.1009
Accuracy of model_2:  0.1952
Accuracy of model_3:  0.1028
Accuracy of model_4:  0.1953
Accuracy of model_5:  0.1945
Accuracy of model_6:  0.1009
Accuracy of model_7:  0.1028
Accuracy of model_8:  0.1959
Accuracy of model_9:  0.1947
Accuracy of model_10:  0.0974


In [16]:
# now lets use them as an ensemble
def predict_ensemble(models, loader):
    # get the pred_proba for each model and average, then take the argmax
    pred_proba = []
    for model in models:
        pred_proba.append(model.predict_proba(loader.dataset.data.float()))
    pred_proba = np.array(pred_proba)
    pred_proba = np.mean(pred_proba, axis=0)
    return np.argmax(pred_proba, axis=1)


y_pred = predict_ensemble(models, test_loader)
accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of vanilla ensemble: ", accuracy)

Accuracy of vanilla ensemble:  0.1961


In [17]:
# now lets try the DES
meta = METADES(models)
# case to double
X = loader_val.dataset.dataset.data.float()
X = X.view(X.size(0), -1).float()  # Flatten the images into vectors
y = loader_val.dataset.dataset.targets.long()
meta.fit(X, y)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# make predictions
X_test = test_loader.dataset.data.float()
X_test = X_test.view(X_test.size(0), -1).float()  # Flatten the images into vectors
y_pred = meta.predict(X_test)
accuracy = np.sum(y_pred == test_loader.dataset.targets.numpy()) / len(
    test_loader.dataset.targets.numpy()
)
print("Accuracy of DES: ", accuracy)

Accuracy of DES:  0.2928
